In [1]:
# Libraries
%matplotlib inline
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from tqdm import tqdm
tqdm.pandas()
pd.options.display.max_colwidth = 150 ###
import numpy as np
import re
import sys
import os
import matplotlib.pyplot as plt
# Add the directory containing visualization_utils.py to path
sys.path.append("/Users/debr/English-Homer/")
import visualization_utils as viz
import seaborn as sns
sns.set_style("whitegrid")
# palette astroblue   orange   genoa      carrot    tawny     neptune      SELAGO    mako   black
color = ['#003D59', '#FD6626','#177070','#FB871D','#641B5E','#86C3BC','#F5E1FD','#414A4F','k']
danB_plotstyle = {'figure.figsize': (12, 7), 
               'axes.labelsize': 'large', # fontsize for x and y labels (was large)
               'axes.titlesize': 'large', # fontsize for title
               'axes.titleweight': 'bold', # font type for title
               'xtick.labelsize': 'large', # fontsize for x 
               'ytick.labelsize':'small', # fontsize fory ticks
               'grid.color': 'k', # grid color
                'grid.linestyle': ':', # grid line style
                'grid.linewidth': 0.2, # grid line width
                'font.family': 'Times New Roman', # font family
                'grid.alpha': 0.5, # transparency of grid
               'figure.dpi': 300, # figure display resolution
               'savefig.bbox': 'tight', # tight bounding box
               'savefig.pad_inches': 0.4, # padding to use when saving
               'axes.titlepad': 15, # title padding
               'axes.labelpad': 8, # label padding
               'legend.borderpad': .6, # legend border padding
               'axes.prop_cycle': plt.cycler(
                color=color) # color cycle for plot lines
               }

# adjust matplotlib defaults
plt.rcParams.update(danB_plotstyle)

In [ ]:
df_G = pd.read_csv(filepath_Green)

# Add translation label
df_W["translation"] = "Wilson"
df_G["translation"] = "Green"

# Keep only necessary columns: book number & tokens
df_W = df_W[["book_num", "tokens", "translation"]]
df_G = df_G[["book_num", "tokens", "translation"]]

# Combine both into one DataFrame
df = pd.concat([df_W, df_G], ignore_index=True)

# Ensure tokens are stored as lists (if stored as strings, convert them)
df["tokens"] = df["tokens"].apply(lambda x: eval(x) if isinstance(x, str) else x)

df

In [4]:
# Load CSVs
filepath_Wilson = "/Users/debr/odysseys_en/Odyssey_dfs/Odyssey_Wilson_eda_END.csv"
filepath_Green = "/Users/debr/odysseys_en/Odyssey_dfs/Odyssey_Green_eda_END.csv"

df_W = pd.read_csv(filepath_Wilson)


In [7]:
from collections import Counter

filepath_Wilson = "/Users/debr/odysseys_en/Odyssey_dfs/Odyssey_Wilson_eda_END.csv"
df_W = pd.read_csv(filepath_Wilson)

# Function to count term frequency correctly
def term_freq_by_doc(list_of_tokens):
    return Counter(eval(list_of_tokens))  # Ensure tokens are treated as a list

# Apply function correctly
df_W["term_freq"] = df_W["tokens"].apply(term_freq_by_doc)

# Display results
df_W[["book_num", "term_freq"]]

,book_num,term_freq
0,1,"{'tell': 14, 'complicated': 1, 'man': 14, 'muse': 1, 'wandered': 1, 'lost': 2, 'wrecked': 1, 'holy': 2, 'town': 3, 'troy': 7, 'went': 10, 'met': 4..."
1,2,"{'dangerous': 1, 'journey': 8, 'early': 1, 'dawn': 2, 'born': 1, 'fingers': 1, 'bloomed': 1, 'odysseus': 18, 'well-beloved': 1, 'son': 11, 'jumped..."
2,3,"{'old': 4, 'king': 8, 'remembers': 1, 'leaving': 1, 'ocean': 1, 'streams': 1, 'sun': 3, 'leapt': 2, 'sky': 1, 'bronze': 2, 'shine': 1, 'light': 3,..."
3,4,"{'sea': 29, 'god': 22, 'said': 11, 'came': 10, 'sparta': 3, 'land': 3, 'caves': 2, 'valleys': 2, 'drove': 2, 'menelaus': 27, 'house': 19, 'found':..."
4,5,"{'goddess': 19, 'storm': 2, 'dawn': 5, 'rose': 5, 'bed': 3, 'lord': 7, 'tithonus': 1, 'bring': 1, 'light': 1, 'deathless': 5, 'gods': 15, 'mortals..."
5,6,"{'princess': 5, 'laundry': 5, 'odysseus': 13, 'suﬀered': 1, 'exhaustion': 1, 'long': 2, 'ordeals': 1, 'hero': 1, 'slept': 1, 'meanwhile': 1, 'athe..."
6,7,"{'magical': 1, 'kingdom': 1, 'odysseus': 14, 'sat': 4, 'patiently': 1, 'prayed': 3, 'meanwhile': 1, 'fine': 6, 'strong': 1, 'mules': 2, 'conveyed'..."
7,8,"{'songs': 2, 'poet': 8, 'soon': 5, 'dawn': 1, 'appeared': 1, 'touched': 2, 'sky': 2, 'roses': 1, 'majestic': 1, 'holy': 2, 'king': 12, 'alcinous':..."
8,9,"{'pirate': 1, 'shepherd': 1, 'cave': 15, 'wily': 1, 'odysseus': 6, 'lord': 7, 'lies': 2, 'answered': 5, 'alcinous': 1, 'great': 6, 'king': 2, 'spl..."
9,10,"{'winds': 5, 'witch': 1, 'reached': 6, 'floating': 1, 'island': 5, 'aeolus': 5, 'well': 2, 'loved': 1, 'deathless': 1, 'gods': 7, 'around': 5, 'sh..."


In [8]:
from collections import Counter

# Function to compute term frequency
def term_freq_by_doc(list_of_tokens):
    token_list = eval(list_of_tokens)  # Ensure tokens are treated as a list
    term_counts = Counter(token_list)  # Count occurrences of each term
    total_terms = len(token_list)  # Total number of terms in the document
    
    # Compute TF: term frequency for each token
    term_freq = {term: count / total_terms for term, count in term_counts.items()}
    
    return term_freq, term_counts

# Apply function to compute TF for each book
df_W["term_freq"], df_W["term_counts"] = zip(*df_W["tokens"].apply(term_freq_by_doc))
#df_W["term_freq"] = df_W["tokens"].apply(term_freq_by_doc)

# Display results
df_W#[["book_num", "term_freq"]].head()

,author,year,title,book_num,first_line,first_sentence,text,num_lines,num_sentences,num_words,scaled_num_words,scaled_num_sentences,scaled_num_lines,tokens,num_tokens,bigrams,term_freq,term_counts
0,Wilson,2017,The Odyssey,1,Tell me about a complicated man.,Tell me about a complicated man,"Tell me about a complicated man.\nMuse, tell me how he wandered and was lost\nwhen he had wrecked the holy town of Troy,\nand where he went, and w...",459,243,3568,-0.554024,-0.557740,-0.615756,"['tell', 'complicated', 'man', 'muse', 'tell', 'wandered', 'lost', 'wrecked', 'holy', 'town', 'troy', 'went', 'met', 'pain', 'suﬀered', 'storms', ...",1730,"['tell complicated', 'complicated man', 'man muse', 'muse tell', 'tell wandered', 'wandered lost', 'lost wrecked', 'wrecked holy', 'holy town', 't...","{'tell': 0.008092485549132947, 'complicated': 0.0005780346820809249, 'man': 0.008092485549132947, 'muse': 0.0005780346820809249, 'wandered': 0.000...","{'tell': 14, 'complicated': 1, 'man': 14, 'muse': 1, 'wandered': 1, 'lost': 2, 'wrecked': 1, 'holy': 2, 'town': 3, 'troy': 7, 'went': 10, 'met': 4..."
1,Wilson,2017,The Odyssey,2,A Dangerous Journey,A Dangerous Journey\nThe early Dawn was born; her fingers bloomed,"A Dangerous Journey\nThe early Dawn was born; her fingers bloomed.\nOdysseus’ well-beloved son\njumped up, put on his clothes, and strapped his sw...",451,260,3463,-0.671121,-0.285542,-0.687191,"['dangerous', 'journey', 'early', 'dawn', 'born', 'fingers', 'bloomed', 'odysseus', 'well-beloved', 'son', 'jumped', 'put', 'clothes', 'strapped',...",1683,"['dangerous journey', 'journey early', 'early dawn', 'dawn born', 'born fingers', 'fingers bloomed', 'bloomed odysseus', 'odysseus well-beloved', ...","{'dangerous': 0.0005941770647653001, 'journey': 0.004753416518122401, 'early': 0.0005941770647653001, 'dawn': 0.0011883541295306002, 'born': 0.000...","{'dangerous': 1, 'journey': 8, 'early': 1, 'dawn': 2, 'born': 1, 'fingers': 1, 'bloomed': 1, 'odysseus': 18, 'well-beloved': 1, 'son': 11, 'jumped..."
2,Wilson,2017,The Odyssey,3,An Old King Remembers,"An Old King Remembers\nLeaving the Ocean’s streams, the Sun leapt up\ninto the sky of bronze, to shine his light\nfor gods and mortals on the fert...","An Old King Remembers\nLeaving the Ocean’s streams, the Sun leapt up\ninto the sky of bronze, to shine his light\nfor gods and mortals on the fert...",512,272,3916,-0.165933,-0.093401,-0.142498,"['old', 'king', 'remembers', 'leaving', 'ocean', 'streams', 'sun', 'leapt', 'sky', 'bronze', 'shine', 'light', 'gods', 'mortals', 'fertile', 'eart...",2031,"['old king', 'king remembers', 'remembers leaving', 'leaving ocean', 'ocean streams', 'streams sun', 'sun leapt', 'leapt sky', 'sky bronze', 'bron...","{'old': 0.0019694731659281144, 'king': 0.003938946331856229, 'remembers': 0.0004923682914820286, 'leaving': 0.0004923682914820286, 'ocean': 0.0004...","{'old': 4, 'king': 8, 'remembers': 1, 'leaving': 1, 'ocean': 1, 'streams': 1, 'sun': 3, 'leapt': 2, 'sky': 1, 'bronze': 2, 'shine': 1, 'light': 3,..."
3,Wilson,2017,The Odyssey,4,What the Sea God Said,"What the Sea God Said\nThey came to Sparta, land of caves and valleys,\nand drove to Menelaus’ house","What the Sea God Said\nThey came to Sparta, land of caves and valleys,\nand drove to Menelaus’ house. They found him\nhosting a wedding feast for ...",865,472,6860,3.117229,3.108934,3.009576,"['sea', 'god', 'said', 'came', 'sparta', 'land', 'caves', 'valleys', 'drove', 'menelaus', 'house', 'found', 'hosting', 'wedding', 'feast', 'many',...",3388,"['sea god', 'god said', 'said came', 'came sparta', 'sparta land', 'land caves', 'caves valleys', 'valleys drove', 'drove menelaus', 'menelaus hou...","{'sea': 0.008559622195985833, 'god': 0.006493506493506494, 'said': 0.003246753246753247, 'came': 0.0029515938606847697, 'sparta': 0.00088547815820...","{'sea': 29, 'god': 22, 'said': 11, 'came': 10, 'sparta': 3, 'land': 3, 'caves': 2, 'valleys': 2, 'drove': 2, 'menelaus': 27, 'house': 19, 'found':..."
4,Wilson,2017